In [1]:

import numpy as np
import math
import scipy.special as scp
import torch
import matplotlib.pyplot as plt
import os

from functions._functions import *
from functions.utils import *
from model.pyr import *
from model.dpwfs import *
from matplotlib.colors import Normalize

wfs = dC()
wfs.wvl = 635
wfs.ps = 3.74e3
wfs.D = 3
wfs.nPx = 128
wfs.modulation = 0
wfs.samp = 2
wfs.rooftop = np.array([0])
wfs.amp_cal = .2
wfs.vNoise = np.array([0,0,0,1])
wfs.fovPx = 2*wfs.samp*wfs.nPx
wfs.magStar = 10**(0.4*(0-0))
wfs.expName = 'Default'
wfs.device ='cuda:0'
#
wfs.zModes = np.array([2,100])
wfs.jModes = torch.arange(wfs.zModes[0], wfs.zModes[1]+1)
#
wfs.pid = os.getpid()
wfs.nHead = 4
wfs.alpha = 3
#
wfs.order = 2
wfs.fw = 'classic'
# DEVICE
wfs.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wfs.precision_name = 'single'
wfs.precision = get_precision(type=wfs.precision_name)

wfs.dx = wfs.D/wfs.nPx# grid spacing of pupil plane
wfs.fovPx = 2*wfs.samp*wfs.nPx  
wfs.magStar = 10**(0.4*(0-wfs.magStar))

wfs.pupil = torch.tensor(CreateTelescopePupil(wfs.nPx))
wfs.pupilLogical = wfs.pupil!=0

wfs.zModes = torch.tensor(wfs.zModes, dtype=wfs.precision.real)
wfs.jModes = torch.arange(wfs.zModes[0], wfs.zModes[1]+1)
wfs.modes = torch.tensor(CreateZernikePolynomials1(wfs)).to(wfs.device).to(wfs.precision.real)
wfs.mInorm = 1
wfs.crop = False
wfs.norm_I = None


pyr = FWFS(wfs)
pyr.nHead = 3
pyr.crop = True

/data2/nhernandez/DPWFS/functions/_functions.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pupil = torch.tensor(wfs.pupil).numpy()


PYR OBJECT INITIALIZED: nPx=128 | nZ=99 | crop=False | M=0 | precision=single


/data2/nhernandez/DPWFS/model/pyr.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.modes = torch.tensor(wfs.modes, dtype=self.precision.real,device=self.device)
/data2/nhernandez/DPWFS/model/pyr.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pupil = torch.tensor(wfs.pupil)# T[N,M]###
/data2/nhernandez/DPWFS/model/pyr.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.pupilLogical = torch.tensor( wfs.pupilLogical, dtype=torch.bool,device=self.device)
/data2/nhernandez/DPWFS/model/functions_model.py:141: UserWarni

# Piston propagations

### c) Piston of models

In [5]:
paper_path_piston = f'./piston'
os.makedirs(paper_path_piston, exist_ok=True)

pyr.pwfs.crop = False
nhs = [2, 3, 4, 4, 5, -1, -5, -10]  # number of sides to be tested
als = [3, 3, 3, 0.2, 3, np.pi/2, np.pi/2, .2]   # corresponding alphas
for idx_ns, (ns, al) in enumerate(zip(nhs,als)):  # correct unpacking
    pyr.pwfs.alpha = al
    pyr.pwfs.nHead = ns
    _,I = pyr(pyr.pwfs.Piston)
    #
    plt.imshow(I.reshape(wfs.nPx, wfs.nPx).detach().cpu())
    plt.axis('off')
    plt.savefig(paper_path_piston + f'/n{ns:.1f}_a{al:.1f}_piston.png', dpi=300, transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()
    #
    plt.imshow( torch.fft.fftshift(torch.angle(pyr.pwfs.fourier_mask.squeeze().detach().cpu())) )
    plt.axis('off')
    plt.savefig(paper_path_piston + f'/fm_n{ns:.1f}_a{al:.1f}.png', dpi=300,transparent=True,bbox_inches='tight',pad_inches=0)
    plt.close()

MAKING A ZWFS
MAKING A ZWFS
MAKING A ZWFS
MAKING A ZWFS


Preconditioners

In [ ]:
paper_path_piston_models = f'./piston_models'
os.makedirs(paper_path_piston_models, exist_ok=True)

# n2
n2 = torch.load('train/single/n2_zN0-nPx128_ns_all_single/routine_0/train_0/Checkpoint/checkpoint_best-v.pth',map_location='cpu').eval()
_,I_n2 = n2(n2.pwfs.Piston)
I_n2 = I_n2.reshape(wfs.nPx,wfs.nPx).detach().cpu()
fm_n2 = torch.fft.fftshift(torch.angle(n2.pwfs.fourier_mask)).detach().cpu().squeeze()
p_n2 = torch.fft.fftshift((n2.DE)).detach().cpu()
fmp_n2 = fm_n2+p_n2
# n3
n3 = torch.load('train/single/n3_zN0-nPx128_ns_all_single/routine_0/train_0/Checkpoint/checkpoint_best-v.pth',map_location='cpu').eval()
_,I_n3 = n3(n3.pwfs.Piston)
I_n3 = I_n3.reshape(wfs.nPx,wfs.nPx).detach().cpu()
fm_n3 = torch.fft.fftshift(torch.angle(n3.pwfs.fourier_mask)).detach().cpu().squeeze()
p_n3 = torch.fft.fftshift((n3.DE)).detach().cpu()
fmp_n3 = fm_n3+p_n3
# n4
n4 = torch.load('train/single/n4_zN0-nPx128_ns_all_single/routine_0/train_0/Checkpoint/checkpoint_best-v.pth',map_location='cpu').eval()
_,I_n4 = n4(n4.pwfs.Piston)
I_n4 = I_n4.reshape(wfs.nPx,wfs.nPx).detach().cpu()
fm_n4 = torch.fft.fftshift(torch.angle(n4.pwfs.fourier_mask)).detach().cpu().squeeze()
p_n4 = torch.fft.fftshift((n4.DE)).detach().cpu()
fmp_n4 = fm_n4+p_n4
# n5
'''n5 = torch.load('train/single/n5_zN0-nPx128_ns_all_single/routine_0/train_0/Checkpoint/checkpoint_best-v.pth',map_location='cpu').eval()
_,I_n5 = n5(n5.pwfs.Piston)
I_n5 = I_n5.reshape(wfs.nPx,wfs.nPx).detach().cpu()
fm_n5 = torch.fft.fftshift(torch.angle(n5.pwfs.fourier_mask)).detach().cpu().squeeze()
p_n5 = torch.fft.fftshift((n5.DE)).detach().cpu()'''
# emptyset
n0 = torch.load('train/single/n0_zN0-nPx128_ns_all_single/routine_0/train_0/Checkpoint/checkpoint_best-v.pth',map_location='cpu').eval()
_,I_n0 = n0(n0.pwfs.Piston)
I_n0 = I_n0.reshape(wfs.nPx,wfs.nPx).detach().cpu()
fm_n0 = torch.fft.fftshift(torch.angle(n0.pwfs.fourier_mask)).detach().cpu().squeeze()
p_n0 = torch.fft.fftshift((n0.DE)).detach().cpu()
fmp_n0 = fm_n0+p_n0
# n4f  
n4f = torch.load('train/single/n4f01_zN0-nPx128_ns_all_single/routine_0/train_0/Checkpoint/checkpoint_best-v.pth',map_location='cpu').eval()
_,I_n4f = n4f(n4f.pwfs.Piston)
I_n4f = I_n4f.reshape(wfs.nPx,wfs.nPx).detach().cpu()
fm_n4f = torch.fft.fftshift(torch.angle(n4f.pwfs.fourier_mask)).detach().cpu().squeeze()
p_n4f = torch.fft.fftshift((n4f.DE)).detach().cpu()
fmp_n4f = fm_n4f+p_n4f
# n4i
n4i = torch.load('train/single/n4i_zN0-nPx128_ns_all_single/routine_0/train_0/Checkpoint/checkpoint_best-v.pth',map_location='cpu').eval()
_,I_n4i = n4i(n4i.pwfs.Piston)
I_n4i = I_n4i.reshape(wfs.nPx,wfs.nPx).detach().cpu()
fm_n4i = torch.fft.fftshift(torch.angle(n4i.pwfs.fourier_mask)).detach().cpu().squeeze()
p_n4i = torch.fft.fftshift((n4i.DE)).detach().cpu()
fmp_n4i = fm_n4i+p_n4i
# Z
z = torch.load('train/single/z_zN0-nPx128_ns_all_single/routine_0/train_0/Checkpoint/checkpoint_best-v.pth',map_location='cpu').eval()
_,I_z = z(z.pwfs.Piston)
I_z = I_z.reshape(wfs.nPx,wfs.nPx).detach().cpu()
fm_z = torch.fft.fftshift(torch.angle(z.pwfs.fourier_mask)).detach().cpu().squeeze()
p_z = torch.fft.fftshift((z.DE)).detach().cpu()
fmp_z = fm_z+p_z
# R
r = torch.load('train/single/r_zN0-nPx128_ns_all_single/routine_0/train_0/Checkpoint/checkpoint_best-v.pth',map_location='cpu').eval()
_,I_r = r(r.pwfs.Piston)
I_r = I_r.reshape(wfs.nPx,wfs.nPx).detach().cpu()
fm_r = torch.fft.fftshift(torch.angle(r.pwfs.fourier_mask)).detach().cpu().squeeze()
p_r = torch.fft.fftshift((r.DE)).detach().cpu()
fmp_r = fm_r+p_r
##
p = [p_n2, p_n3, p_n4, p_n0, p_n4f, p_n4i, p_z, p_r]
I = [I_n2,I_n3,I_n4,I_n0,I_n4f,I_n4i,I_z,I_r]
fm = [fm_n2,fm_n3,fm_n4,fm_n0,fm_n4f,fm_n4i,fm_z,fm_r]
fmp = [fmp_n2,fmp_n3,fmp_n4,fmp_n0,fmp_n4f,fmp_n4i,fmp_z,fmp_r]
dpi = 300
for idx_n in range(len(p)):
    # piston
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    im=ax.imshow( I[idx_n], cmap='hot' )
    plt.axis('off')
    plt.savefig(paper_path_piston_models + f'/I_mo{idx_n}_dpi{dpi}.png', dpi=dpi, transparent=True, bbox_inches='tight', pad_inches=0)
    plt.savefig(paper_path_piston_models + f'/I_mo{idx_n}_dpi{dpi}.pdf', dpi=dpi, transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()
    # preconditioner
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    im=ax.imshow( p[idx_n], cmap='hsv', vmin=-np.pi, vmax=np.pi )
    plt.axis('off')
    plt.savefig(paper_path_piston_models + f'/p_mo{idx_n}_dpi{dpi}.png', dpi=dpi, transparent=True, bbox_inches='tight', pad_inches=0)
    plt.savefig(paper_path_piston_models + f'/p_mo{idx_n}_dpi{dpi}.pdf', dpi=dpi, transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()
    # fourier mask
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    im = ax.imshow(fm[idx_n].squeeze(), vmin=-np.pi, vmax=np.pi, cmap='hsv')
    plt.axis('off')
    '''if idx_n == len(p)-1: 
        pos = ax.get_position()
        cbar_ax = fig.add_axes([pos.x1 -.4, pos.y0, .5, pos.height])
        cbar = fig.colorbar(im, ax=cbar_ax)
        plt.axis('off')
        cbar.set_ticks([-np.pi,0,np.pi])
        cbar.set_ticklabels([r'$-\pi$',r'$0$', r'$\pi$'])
        cbar.ax.tick_params(labelsize=25)'''
        #cbar.ax.set_aspect(10) 
    plt.savefig(paper_path_piston_models+f'/fm_mo{idx_n}_dpi{dpi}.png',dpi=dpi,transparent=True, bbox_inches='tight', pad_inches=0)
    plt.savefig(paper_path_piston_models+f'/fm_mo{idx_n}_dpi{dpi}.pdf',dpi=dpi,transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()  
    # fourier mask + preconditioner  
    fig,ax = plt.subplots(1,1,figsize=(5,5))
    im = ax.imshow(fmp[idx_n], vmin=-np.pi, vmax=np.pi, cmap='hsv')
    plt.axis('off')
        #cbar.ax.set_aspect(10) 
    plt.savefig(paper_path_piston_models+f'/fmp_mo{idx_n}_dpi{dpi}.png',dpi=dpi,transparent=True, bbox_inches='tight', pad_inches=0)
    plt.savefig(paper_path_piston_models+f'/fmp_mo{idx_n}_dpi{dpi}.pdf',dpi=dpi,transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()

    # only for colorbar
    if idx_n == len(p)-1: 
        fig,ax = plt.subplots(1,1,figsize=(5,5))
        im = ax.imshow(fm[idx_n].squeeze(), vmin=-np.pi, vmax=np.pi, cmap='hsv')
        plt.axis('off')
        pos = ax.get_position()
        cbar_ax = fig.add_axes([pos.x1 -.4, pos.y0, .5, pos.height])
        cbar = fig.colorbar(im, ax=cbar_ax)
        plt.axis('off')
        cbar.set_ticks([-np.pi,0,np.pi])
        cbar.set_ticklabels([r'$-\pi$',r'$0$', r'$\pi$'])
        cbar.ax.tick_params(labelsize=25)
        plt.savefig(paper_path_piston_models+f'/colorbar_dpi{dpi}.pdf',dpi=dpi,transparent=True, bbox_inches='tight', pad_inches=0)
        plt.close()

/data2/nhernandez/miniconda3/envs/DPWFS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data2/nhernandez/DPWFS/model/functions_model.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  npv = torch.tensor( (obj.fovPx-sx)/2, dtype=obj.precision.int)
